In [115]:
import pandas as pd
import sklearn.ensemble
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn import datasets, linear_model
from sklearn.preprocessing import StandardScaler
import numpy as np
import time
import datetime

In [116]:
features = pd.read_csv('./features.csv', index_col='match_id')
Y = features[['duration', 'radiant_win','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire']]
X = features.drop(columns=['duration', 'radiant_win','tower_status_radiant','tower_status_dire','barracks_status_radiant','barracks_status_dire'])

X_test = pd.read_csv('./features_test.csv', index_col='match_id')
features_test = X_test

Всего матчей 97230. Ниже представлены признаки с пропусками и их количество.

In [117]:
for column in X:
    if X[column].count() < 97230:
        print(column + ":", end = " ")
        print(97230 - X[column].count())

first_blood_time: 19553
first_blood_team: 19553
first_blood_player1: 19553
first_blood_player2: 43987
radiant_bottle_time: 15691
radiant_courier_time: 692
radiant_flying_courier_time: 27479
radiant_first_ward_time: 1836
dire_bottle_time: 16143
dire_courier_time: 676
dire_flying_courier_time: 26098
dire_first_ward_time: 1826


In [118]:
X = X.fillna(0)
X = pd.DataFrame(StandardScaler().fit_transform(X), columns=X.columns, index=X.index)

X_test = X_test.fillna(0)
X_test = pd.DataFrame(StandardScaler().fit_transform(X_test), columns=X_test.columns, index=X_test.index)

In [119]:
y = Y["radiant_win"]

1. Оцените качество логистической регрессии (sklearn.linear_model.LogisticRegression с L2-регуляризацией) с помощью кросс-валидации по той же схеме, которая использовалась для градиентного бустинга. Подберите при этом лучший параметр регуляризации (C).

In [120]:
kf = sklearn.model_selection.KFold(n_splits=5, shuffle=True)

param_grid = {
    'C' : [0.00001, 0.0001, 0.0001, 0.001, 0.01, 0.1, 1]
}
clf = linear_model.LogisticRegression()
best_clf = GridSearchCV(clf, param_grid)
best_clf.fit(X, y)
print(best_clf.best_params_)
start_time = datetime.datetime.now()
score_1 = cross_val_score(best_clf.best_estimator_, X, y, cv=kf, scoring='roc_auc')
print(np.mean(score_1))
print('Time elapsed:', datetime.datetime.now() - start_time)

{'C': 0.001}
0.7162889919352595
Time elapsed: 0:00:10.009683


2. Среди признаков в выборке есть категориальные, которые мы использовали как числовые, что вряд ли является хорошей идеей. Категориальных признаков в этой задаче одиннадцать: lobby_type и r1_hero, r2_hero, ..., r5_hero, d1_hero, d2_hero, ..., d5_hero. Уберите их из выборки, и проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Изменилось ли качество? Чем вы можете это объяснить?

In [121]:
X_d = X.drop(columns=['lobby_type', 'r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero'])

X_test = X_test.drop(columns=['lobby_type', 'r1_hero','r2_hero','r3_hero','r4_hero','r5_hero','d1_hero','d2_hero','d3_hero','d4_hero','d5_hero'])

In [122]:
best_clf_2 = GridSearchCV(clf, param_grid)
best_clf_2.fit(X_d, y)
print(best_clf_2.best_params_)
score_2 = cross_val_score(best_clf_2.best_estimator_, X_d, y, cv=kf, scoring='roc_auc')
print(np.mean(score_2))

{'C': 0.001}
0.7161436995991657


3. На предыдущем шаге мы исключили из выборки признаки rM_hero и dM_hero, которые показывают, какие именно герои играли за каждую команду. Это важные признаки — герои имеют разные характеристики, и некоторые из них выигрывают чаще, чем другие. Выясните из данных, сколько различных идентификаторов героев существует в данной игре (вам может пригодиться фукнция unique или value_counts).

In [123]:
np.count_nonzero(np.unique(features['d1_hero']))

108

4. Воспользуемся подходом "мешок слов" для кодирования информации о героях. Пусть всего в игре имеет N различных героев. Сформируем N признаков, при этом i-й будет равен нулю, если i-й герой не участвовал в матче; единице, если i-й герой играл за команду Radiant; минус единице, если i-й герой играл за команду Dire. Ниже вы можете найти код, который выполняет данной преобразование. Добавьте полученные признаки к числовым, которые вы использовали во втором пункте данного этапа.

In [124]:
hero_D = np.unique(features['d1_hero'])
N = max(hero_D)
X_pick = np.zeros((features.shape[0], N))
for i, match_id in enumerate(features.index):
    for p in range(5):
        X_pick[i, features.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick[i, features.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1

X_pick_test = np.zeros((features_test.shape[0], N))
for i, match_id in enumerate(features_test.index):
    for p in range(5):
        X_pick_test[i, features_test.ix[match_id, 'r%d_hero' % (p+1)]-1] = 1
        X_pick_test[i, features_test.ix[match_id, 'd%d_hero' % (p+1)]-1] = -1


D:\work\Programms\Programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  
D:\work\Programms\Programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  import sys
D:\work\Programms\Programming\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  if sys.path[0] == '':
D:\work\Programms\Programming\An

In [125]:
heroes = pd.DataFrame(X_pick)
heroes = heroes.astype(int)
heroes.index = X_d.index
X_full = pd.merge(X_d, heroes, on='match_id', how="inner")

heroes2 = pd.DataFrame(X_pick_test)
heroes2 = heroes2.astype(int)
heroes2.index = X_test.index
X_full_test = pd.merge(X_test, heroes2, on='match_id', how="inner")

5. Проведите кросс-валидацию для логистической регрессии на новой выборке с подбором лучшего параметра регуляризации. Какое получилось качество? Улучшилось ли оно? Чем вы можете это объяснить?

In [126]:
clf_C = linear_model.LogisticRegression(C = 0.001)
score_3 = cross_val_score(clf_C, X_full, y, cv=kf, scoring='roc_auc')
print(np.mean(score_3))

0.7461970051691635


6. Постройте предсказания вероятностей победы команды Radiant для тестовой выборки с помощью лучшей из изученных моделей (лучшей с точки зрения AUC-ROC на кросс-валидации). Убедитесь, что предсказанные вероятности адекватные — находятся на отрезке [0, 1], не совпадают между собой (т.е. что модель не получилась константной).

In [128]:
clf_C.fit(X_full, y)
pred = clf_C.predict_proba(X_full_test)[:, 1]
print("min:", min(pred))
print("max:", max(pred))

min: 0.0098705566002187
max: 0.9924174814067266
